In [1]:
import time; start_time=time.time()

In [2]:
import numpy as np
import pandas as pd
from sklearn import ensemble
import xgboost as xgb
from sklearn.metrics import log_loss,make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import random; random.seed(2016)

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [4]:
#Number of rows in training data
num_train = train.shape[0]

In [5]:
#Save target from train in another series and Drop the target variable 
y_train = train['target']
train = train.drop('target', axis=1)
id_test = test['ID']

In [6]:
#Merge the train and test data to perform operations on the data
df_all = pd.concat((train,test),axis=0,ignore_index=True)
df_all['null_count'] = df_all.isnull().sum(axis=1).tolist()


In [7]:
#Drop the ids
df_all_temp = df_all['ID']
df_all = df_all.drop('ID', axis=1)
d_col_drops = []

In [8]:
def fill_nan_null(val):
    ret_fill_nan_null = 0.0
    if val==True:
        ret_fill_nan_null = 1.0
    return ret_fill_nan_null

In [9]:
# Function to convert to hexavigesimal base..Special treamtement to v22
def az_to_int(az,nanVal=None):
    if az==az:  #catch NaN
        hv = 0
        for i in range(len(az)):
            hv += (ord(az[i].lower())-ord('a')+1)*26**(len(az)-1-i)
        return hv
    else:
        if nanVal is not None:
            return nanVal
        else:
            return az

# convert v22 to hexavigesimal
df_all['v22'] = df_all.v22.apply(az_to_int)

In [10]:
df_data_types = df_all.dtypes[:]
for i in range(len(df_data_types)):
    df_all[str(df_data_types.index[i])+'_nan_'] = df_all[str(df_data_types.index[i])].map(lambda x:fill_nan_null(pd.isnull(x)))

In [11]:
df_all = df_all.fillna(-1)

In [12]:
for i in range(len(df_data_types)):
    if(str(df_data_types[i]) == 'object'):
        df_u = pd.unique(df_all[str(df_data_types.index[i])].ravel())
        print("Column: ", str(df_data_types.index[i]), " Length: ", len(df_u))
        d={}
        df_all[str(df_data_types.index[i])], tmpItter = df_all[str(df_data_types.index[i])].factorize()
        d_col_drops.append(str(df_data_types.index[i]))
        if len(df_u)<150:
            dummies = pd.get_dummies(df_all[str(df_data_types.index[i])],prefix=str(df_data_types.index[i])) #.rename(columns = lambda x: str(df_data_types.index[i])+str(x))
            df_all_temp = pd.concat([df_all_temp,dummies], axis=1)

Column:  v3  Length:  4
Column:  v24  Length:  5
Column:  v30  Length:  8
Column:  v31  Length:  4
Column:  v47  Length:  10
Column:  v52  Length:  13
Column:  v56  Length:  131
Column:  v66  Length:  3
Column:  v71  Length:  12
Column:  v74  Length:  3
Column:  v75  Length:  4
Column:  v79  Length:  18
Column:  v91  Length:  8
Column:  v107  Length:  8
Column:  v110  Length:  3
Column:  v112  Length:  23
Column:  v113  Length:  38
Column:  v125  Length:  91


In [13]:
df_all_temp = df_all_temp.drop('ID',axis=1)
df_all = pd.concat([df_all,df_all_temp], axis=1)
print(len(df_all),len(df_all.columns))

228714 650


In [14]:
#Just in case, we need to start all the things over again
#df_all.to_csv("df_all.csv")

In [15]:
d_col_drops

['v3',
 'v24',
 'v30',
 'v31',
 'v47',
 'v52',
 'v56',
 'v66',
 'v71',
 'v74',
 'v75',
 'v79',
 'v91',
 'v107',
 'v110',
 'v112',
 'v113',
 'v125']

In [16]:
#Seperate out train and test data to run the model
train = df_all.iloc[:num_train]
test = df_all.iloc[num_train:]
train = train.drop(d_col_drops,axis=1)
test = test.drop(d_col_drops,axis=1)

In [17]:
df_all.columns.values

array(['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11',
       'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20',
       'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29',
       'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38',
       'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47',
       'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56',
       'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65',
       'v66', 'v67', 'v68', 'v69', 'v70', 'v71', 'v72', 'v73', 'v74',
       'v75', 'v76', 'v77', 'v78', 'v79', 'v80', 'v81', 'v82', 'v83',
       'v84', 'v85', 'v86', 'v87', 'v88', 'v89', 'v90', 'v91', 'v92',
       'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101',
       'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v108', 'v109',
       'v110', 'v111', 'v112', 'v113', 'v114', 'v115', 'v116', 'v117',
       'v118', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v125',
       'v1

In [18]:
def flog_loss(ground_truth,predictions):
    flog_loss_ = log_loss(ground_truth,predictions)
    return flog_loss_

LL = make_scorer(flog_loss, greater_is_better=False)

In [19]:
#g={'ne':150,'md':6,'mf':80,'rs':2016} 
#Suggested one 
g={'ne':500,'md':40,'mf':60,'rs':2016}
etc = ensemble.ExtraTreesClassifier(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], 
                                    random_state=g['rs'], criterion='entropy', min_samples_split= 4, 
                                    min_samples_leaf= 2, verbose = 0, n_jobs =-1)

etr = ensemble.ExtraTreesRegressor(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], 
                                   random_state=g['rs'], min_samples_split= 4, min_samples_leaf= 2,
                                   verbose = 0, n_jobs =-1)    

rfc = ensemble.RandomForestClassifier(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], 
                                      random_state=g['rs'], criterion='entropy', min_samples_split= 4, 
                                      min_samples_leaf= 2, verbose = 0, n_jobs =-1)

rfr = ensemble.RandomForestRegressor(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], 
                                     random_state=g['rs'], min_samples_split= 4, min_samples_leaf= 2,
                                     verbose = 0, n_jobs =-1)

xgr = xgb.XGBRegressor(n_estimators=g['ne'], max_depth=g['md'], seed=g['rs'], missing=np.nan, 
                       learning_rate=0.01, subsample=0.9, colsample_bytree=0.85, 
                       objective='reg:linear')

xgc = xgb.XGBClassifier(n_estimators=g['ne'], max_depth=g['md'], seed=g['rs'], missing=np.nan, 
                        learning_rate=0.01, subsample=0.9, colsample_bytree=0.85, 
                        objective='binary:logistic') #try 'binary:logitraw'


In [20]:
clf = {'etc':etc, 'etr':etr, 'rfc':rfc, 'rfr':rfr, 'xgr':xgr, 'xgc':xgc} # use this line instead
# removed due to kaggle performance, would prefer less time and more cores than more time and less cores :)
#clf = {'etr':etr, 'rfr':rfr, 'xgr':xgr}
#Since i am getting a pickling error, we moved to only xgb regressor and classifier
#clf = {'xgr':xgr,'xgc':xgc}

In [21]:
y_pred=[]
best_score = 0.0
id_results = id_test[:]

In [22]:
for c in clf:
    if c[:1] != "x": #not xgb
        model = GridSearchCV(estimator=clf[c], param_grid={}, n_jobs=1, cv=2, verbose=0, scoring=LL)
        model.fit(train, y_train.values)
        if c[-1:] != "c": #not classifier
            y_pred = model.predict(test)
            print("Ensemble Model: ", c, " Best CV score: ", model.best_score_, " Time: ", round(((time.time() - start_time)/60),2))
        else: #classifier
            best_score = (log_loss(y_train.values, model.predict_proba(train)))*-1
            y_pred = model.predict_proba(test)[:,1]
            print("Ensemble Model: ", c, " Best CV score: ", best_score, " Time: ", round(((time.time() - start_time)/60),2))
    else: #xgb
        #X_fit, X_eval, y_fit, y_eval= train_test_split(train, y_train, test_size=0.35, train_size=0.65, random_state=g['rs'])
        
        model = clf[c]
        
        #Make some arrangements for cross validation of xgboost
        xgb_param = model.get_xgb_params()
        xgtrain = xgb.DMatrix(train.values, label=y_train.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=model.get_params()['n_estimators'], nfold=5,
            metrics='logloss', early_stopping_rounds=20, verbose_eval=1)
        
        model.set_params(n_estimators=cvresult.shape[0])
        #model.fit(X_fit, y_fit.values, early_stopping_rounds=20, eval_metric="logloss", eval_set=[(X_eval, y_eval)], verbose=0)
        model.fit(train, y_train.values, eval_metric="logloss", verbose=0)
        if c == "xgr": #xgb regressor
            best_score = (log_loss(y_train.values, model.predict(train)))*-1
            y_pred = model.predict(test)
        else: #xgb classifier
            best_score = (log_loss(y_train.values, model.predict_proba(train)))*-1
            y_pred = model.predict_proba(test)[:,1]
        print("Ensemble Model: ", c, " Best CV score: ", best_score, " Time: ", round(((time.time() - start_time)/60),2))
    for i in range(len(y_pred)):
        if y_pred[i]<0.0:
            y_pred[i] = 0.0
        if y_pred[i]>1.0:
            y_pred[i] = 1.0
    df_in = pd.DataFrame({"ID": id_test, c: y_pred})
    id_results = pd.concat([id_results, df_in[c]], axis=1)
        
id_results['avg'] = id_results.drop('ID', axis=1).apply(np.average, axis=1)
id_results['min'] = id_results.drop('ID', axis=1).apply(min, axis=1)
id_results['max'] = id_results.drop('ID', axis=1).apply(max, axis=1)
id_results['diff'] = id_results['max'] - id_results['min']
for i in range(10):
    print(i, len(id_results[id_results['diff']>(i/10)]))
id_results.to_csv("results_analysis.csv", index=False)
ds = id_results[['ID','avg']]
ds.columns = ['ID','PredictedProb']
ds.to_csv('submission.csv',index=False)

Will train until cv error hasn't decreased in 20 rounds.
[0]	cv-test-logloss:0.6892142+6.0680804213510756e-05	cv-train-logloss:0.6849982+3.2208073522006e-05
[1]	cv-test-logloss:0.6854364+7.331193627231324e-05	cv-train-logloss:0.6768908+7.41145060025293e-05
[2]	cv-test-logloss:0.6816827999999999+0.00012111713338747449	cv-train-logloss:0.6689693999999999+7.386095044068349e-05
[3]	cv-test-logloss:0.6780544+0.00010735101303666865	cv-train-logloss:0.661157+8.224840423984264e-05
[4]	cv-test-logloss:0.6744810000000001+0.00011280957406178287	cv-train-logloss:0.6535105999999999+5.7579857589260984e-05
[5]	cv-test-logloss:0.670978+0.0001786896751354082	cv-train-logloss:0.645974+6.864400920690693e-05
[6]	cv-test-logloss:0.6675114000000001+0.00018931941263378732	cv-train-logloss:0.6385202+5.356640738372625e-05
[7]	cv-test-logloss:0.6641292+0.00016074003857158197	cv-train-logloss:0.6312336000000001+6.383290687412887e-05
[8]	cv-test-logloss:0.6608306+0.00021608572373016684	cv-train-logloss:0.62406719

Ensemble Model:  xgr  Best CV score:  -0.033876200369  Time:  291.49
Ensemble Model:  etc  Best CV score:  -0.152550052476  Time:  311.18
Ensemble Model: 

Stopping. Best iteration:
[359] cv-mean:0.47134960000000004	cv-std:0.002094424656080998
Will train until cv error hasn't decreased in 20 rounds.
[0]	cv-test-logloss:0.6893748000000001+3.604663645890191e-05	cv-train-logloss:0.6870616+9.769339793478122e-06
[1]	cv-test-logloss:0.685706+5.7469992169818805e-05	cv-train-logloss:0.681212+4.291386722262161e-05
[2]	cv-test-logloss:0.6820958+6.545655047434146e-05	cv-train-logloss:0.6754903999999999+6.881743965013614e-05
[3]	cv-test-logloss:0.6786014+4.341244061328762e-05	cv-train-logloss:0.6698487999999999+0.00012933584190007101
[4]	cv-test-logloss:0.6751366000000001+9.333938075646577e-05	cv-train-logloss:0.6642888+0.00017672962400228957
[5]	cv-test-logloss:0.6717772+0.00012495023009185743	cv-train-logloss:0.6588566+0.00015220722716089268
[6]	cv-test-logloss:0.6684348+0.0001275670803930421	cv-train-logloss:0.653449+0.0001879233886454807
[7]	cv-test-logloss:0.6651659999999999+0.000120445838450309	cv-train-logloss:0.6481086+0.00023543117890375114


 rfc  Best CV score:  -0.184324511987  Time:  332.02
Ensemble Model:  xgc  Best CV score:  -0.149849165678  Time:  519.44
Ensemble Model:  etr  Best CV score:  -0.472777637258  Time:  537.28
Ensemble Model:  rfr  Best CV score:  -0.47152500297  Time:  551.54
0 114393
1 56224
2 13174
3 3676
4 1105
5 266
6 58
7 6
8 0
9 0


Stopping. Best iteration:
[279] cv-mean:0.467155	cv-std:0.0020970669040352597
